In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
formats = {
    ('integer', 2): 'H',
    ('integer', 4): 'I',
    ('integer', 6): '6s',
    ('integer', 8): 'Q',
    ('alpha',   1): 's',
    ('alpha',   2): '2s',
    ('alpha',   4): '4s',
    ('alpha',   8): '8s',
    ('price_4', 4): 'I',
    ('price_8', 8): 'Q',
}

In [ ]:
import sys
import os
import gzip
import pandas as pd
import struct

stock_map = dict()
itch_p_size = { "S": 11, "R": 38, "H": 24, "Y": 19, "L": 25,
                "A": 35, "F": 39, "E": 30, "C": 35, "X": 22,
                "D": 18, "U": 34, "P": 43, "Q": 39, "B": 18,
                "V": 34, "W": 11, "K": 27, "J": 34, "h": 20,
                "I": 49, "N": 19}

def genTestData(file):
    m_type = file.read(1)
    started = False # Tracks if the market opened yet

    order_cnt=0
    total_cnt=10000

    with open('nsdaq_itch50_08302019_0.txt', 'w') as f:
      f.write("")

    while m_type:
        m_type = m_type.decode()
        if m_type in itch_p_size.keys():
            msg = file.read(itch_p_size[m_type])

            if m_type == "S":
                data = struct.unpack('>HH6sc',msg)
                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                event_code   = data[3].decode().strip()
                print(m_type, stock_locate, tracking_num, timestamp, event_code)
                with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                  print(m_type, stock_locate, tracking_num, timestamp, event_code, file=f)
                if data[3].decode() == "Q": # Start of Market hours
                    openTime = timeToInt(data[2])
                    print("Market opened at %d nanoseconds: " % openTime)
                    started = True
                elif data[3].decode() == "M": # End of Market hours
                    endTime = timeToInt(data[2])
                    print("Market closed at %d nanoseconds: " % endTime)
                    break
            elif m_type == "R":
                data = struct.unpack('>HH6s8sccIcc2scccccIc',msg)
                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ticker_symbol = data[3].decode().strip()


                if(ticker_symbol=="AAPL" or
                   ticker_symbol=="MSFT" or
                   ticker_symbol=="NVDA" or
                   ticker_symbol=="GOOG" or
                   ticker_symbol=="AMZN" or
                   ticker_symbol=="FB"   or
                   ticker_symbol=="NFLX" or
                   ticker_symbol=="AMD"  or
                   ticker_symbol=="INTC" or
                   ticker_symbol=="COST" or
                   ticker_symbol=="JPM"  or
                   ticker_symbol=="TSLA"):
                  stock_map[stock_locate] = ticker_symbol
                  print(m_type, stock_locate, tracking_num, timestamp, ticker_symbol)
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ticker_symbol, file=f)


            elif m_type == "A" and started == True:
                data = struct.unpack('>HH6sQcI8sI',msg)

                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ord_ref_num  = data[3]
                order_side   = data[4].decode().strip()
                num_shares   = data[5]
                ticker_symbol= data[6].decode().strip()
                price        = data[7]

                if stock_locate in stock_map.keys():
                  order_cnt += 1
                  print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, order_side, num_shares, ticker_symbol, price)
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, order_side, num_shares, ticker_symbol, price, file=f)
                  if(order_cnt>=total_cnt):
                    break

            elif m_type == "F" and started == True:
                data = struct.unpack('>HH6sQcI8sI4s',msg)

                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ord_ref_num  = data[3]
                order_side   = data[4].decode().strip()
                num_shares   = data[5]
                ticker_symbol= data[6].decode().strip()
                price        = data[7]
                attribution  = data[8].decode().strip()

                if stock_locate in stock_map.keys():
                  order_cnt += 1
                  print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, order_side, num_shares, ticker_symbol, price, attribution)
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, order_side, num_shares, ticker_symbol, price, attribution, file=f)
                  if(order_cnt>=total_cnt):
                    break

            elif m_type == "E" and started == True:
                data = struct.unpack('>HH6sQIQ',msg)

                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ord_ref_num  = data[3]
                num_shares   = data[4]
                match_num    = data[5]

                if stock_locate in stock_map.keys():
                  order_cnt += 1
                  print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, num_shares, match_num)
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, num_shares, match_num, file=f)
                  if(order_cnt>=total_cnt):
                    break

            elif m_type == "C" and started == True:
                data = struct.unpack('>HH6sQIQcI',msg)

                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ord_ref_num  = data[3]
                num_shares   = data[4]
                match_num    = data[5]
                printable    = data[6].decode().strip()
                exe_price    = data[7]

                if stock_locate in stock_map.keys():
                  order_cnt += 1
                  print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, num_shares, match_num, printable, exe_price)
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, num_shares, match_num, printable, exe_price, file=f)
                  if(order_cnt>=total_cnt):
                    break

            elif m_type == "X" and started == True:
                data = struct.unpack('>HH6sQI',msg)

                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ord_ref_num  = data[3]
                num_shares   = data[4]

                if stock_locate in stock_map.keys():
                  order_cnt += 1
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, num_shares, file=f)
                  print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, num_shares)
                  if(order_cnt>=total_cnt):
                    break

            elif m_type == "D" and started == True:
                data = struct.unpack('>HH6sQ',msg)

                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ord_ref_num  = data[3]

                if stock_locate in stock_map.keys():
                  order_cnt += 1
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, file=f)
                  print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num)
                  if(order_cnt>=total_cnt):
                    break

            elif m_type == "U" and started == True:
                data = struct.unpack('>HH6sQQII',msg)

                stock_locate = data[0]
                tracking_num = data[1]
                timestamp    = timeToInt(data[2])
                ord_ref_num  = data[3]
                new_ref_num  = data[4]
                num_shares   = data[5]
                price        = data[6]

                if stock_locate in stock_map.keys():
                  order_cnt += 1
                  with open('nsdaq_itch50_08302019_0.txt', 'a') as f:
                    print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, new_ref_num, num_shares, price, file=f)
                  print(m_type, stock_locate, tracking_num, timestamp, ord_ref_num, new_ref_num, num_shares, price)
                  if(order_cnt>=total_cnt):
                    break

        m_type = file.read(1)
    return

def timeToInt(timestamp):
    toIntegerWidth = struct.pack('>2s6s',b'\x00\x00', timestamp)
    timeInInt = struct.unpack('>Q', toIntegerWidth)
    return timeInInt[0]

# Add the nasdaq .gz file to drive and mount the drive first*
fileName = '/content/drive/MyDrive/08302019.NASDAQ_ITCH50.gz'
print("Input NASDAQ file %s: " % fileName)
file = gzip.open(fileName, 'rb')
genTestData(file)
print("Created the .txt file!")


Input NASDAQ file /content/drive/MyDrive/08302019.NASDAQ_ITCH50.gz: 
S 0 0 11046036981912 O
R 14 0 11316046568183 AAPL
R 339 0 11316059936969 AMD
R 389 0 11316061686253 AMZN
R 1635 0 11316093790700 COST
R 2731 0 11316118177636 FB
R 3448 0 11316133653951 GOOG
R 4173 0 11316149072417 INTC
R 4480 0 11316155922445 JPM
R 5277 0 11316172769291 MSFT
R 5487 0 11316177424218 NFLX
R 5695 0 11316182008965 NVDA
R 7922 0 11316229591151 TSLA
S 0 0 14400000191772 S
S 0 0 34200000081775 Q
Market opened at 34200000081775 nanoseconds: 
A 7922 0 34200000557033 11222404 S 100 TSLA 2321900
A 7922 0 34200000944762 11222432 B 100 TSLA 2271100
D 7922 0 34200001201078 9573336
D 339 0 34200001452372 5764565
D 339 0 34200001765560 5764569
F 339 0 34200001774045 7940409 B 100 AMD 229300 SBSH
F 339 0 34200002254808 7940637 S 100 AMD 404700 SBSH
A 4480 0 34200002509113 5841798 S 100 JPM 1100000
D 5277 0 34200003100568 7542903
A 5695 0 34200003111669 7549859 B 100 NVDA 1686800
A 5695 0 34200003113557 7549863 B 100 N

In [13]:
stock_map

{14: 'AAPL',
 339: 'AMD',
 389: 'AMZN',
 1635: 'COST',
 2731: 'FB',
 3448: 'GOOG',
 4173: 'INTC',
 4480: 'JPM',
 5277: 'MSFT',
 5487: 'NFLX',
 5695: 'NVDA',
 7922: 'TSLA'}